                                                                    بِسْمِ اللَّـهِ الرَّحْمَـٰنِ الرَّحِيمِ 


                                                                        Google Maps Scrapper

In [3]:
from selenium import webdriver
#import pyautogui
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException


In [4]:
filename = "Cairo resturants"
link = "https://www.google.com/maps/search/resturants+in+cairo/@30.261226,31.3748331,11z/data=!3m1!4b1?entry=ttu" 
# browser = webdriver.Edge()
record = []
e = []
le = 0

driver=webdriver.Chrome()

def Selenium_extractor():
    

    # Change language to English  
    langChangeStep1=driver.find_element(By.XPATH,""" //*[@id="QA0Szd"]/div/div/div[1]/div[1]/ul/li[1]/button """)
    langChangeStep1.click()
    time.sleep(2)
    langChangeStep2=driver.find_element(By.XPATH,"""//*[@id="settings"]/div/div[2]/ul/div[7]/li[1]/button""")
    langChangeStep2.click()
    time.sleep(2)
    #langChangeStep3=driver.find_element(By.XPATH,""" //*[@id="languages"]/div/div[2]/div/div/a""")
    #langChangeStep3=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[3]/div/div/div/div[2]/div/div/a """)
    langChangeStep3=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div/a """)
    langChangeStep3.click()
    time.sleep(2)

    action = ActionChains(driver)
    a = driver.find_elements(By.CLASS_NAME, "hfpxzc")


    while len(a) < 55:
        print(len(a))
        var = len(a)
        scroll_origin = ScrollOrigin.from_element(a[len(a)-1])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        time.sleep(2)
        a = driver.find_elements(By.CLASS_NAME, "hfpxzc")

        if len(a) == var:
            #The (le) variable is used to avoid weak network data loading 
            #it gives another chance for the elements to load 
            le+=1
            if le > 20:
                break
        else:
            le = 0
    print("----------------------")
    print("    Scrolling Done    ")
    print("----------------------")
    
    
    for i in range(len(a)):
        
        a = driver.find_elements(By.CLASS_NAME, "hfpxzc")
        scroll_origin = ScrollOrigin.from_element(a[i])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        action.move_to_element(a[i]).perform()
        
        #-----------------------------------------------------------------------------------------
        p1 = driver.current_window_handle                #(save the last place in the main page)
        #------------------------------------------------------------------------------------------    
        t1="(//a[@class='hfpxzc'])["
        t2=i+1
        t3="]"
        aPath=str(t1)+str(t2)+str(t3)
        aLink=driver.find_element(By.XPATH,str(aPath)).get_attribute("href")
        # Open a new window
        driver.execute_script("window.open('');")
        # Switch to the new window and open the website
        driver.switch_to.window(driver.window_handles[1])
        driver.get(aLink)
        time.sleep(2)
        

        #This (try-except) is done to ensure that the element is loaded so its clickable or not to avoid any (element not clickable)error
        # while(True):
        #     try:
        #         a[i].click()
        #         break
        #     except:
        #         continue

        time.sleep(2)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        try:
        
            #name
            Name_Html = soup.findAll('h1', {"class": "DUwDvf lfPIob"})
            name = Name_Html[0].text
            if name not in e:
                e.append(name)
                divs = soup.findAll('div', {"class": "Io6YTe fontBodyMedium kR99db"})
                print(name)
                
                #photos:
                # Cases to be handled:
                # 1- no any photos (the default blue photo will be found and should be ignored)
                # 2- only one photo found
                # 3- photos numbers in between 1 and 13
                # 4- photos more than 13
                # 5- only one photo but written more than that number ie.(2 photos)
                
                
                Photos = [None] * 20
                try:
                    time.sleep(2)
                    photoLinkButton=driver.find_element(By.XPATH ,""" //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button/img """)

                    #this try-except aim is to handle if their is no more than one photo
                    try:
                        validateIfOnePhotoOnly=driver.find_element(By.XPATH,""" //button[@class="Dx2nRe"]//div """)
                    except:
                        Photos[0]=driver.find_element(By.XPATH,""" //div[@class="RZ66Rb FgCUCc"]//img """).get_attribute("src")
                        fakeCodeToExitTheOuterTryExcept=driver.find_element(By.XPATH,"""  //:)  """)

                    photoLinkButton=driver.find_element(By.XPATH ,""" //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button/img  """)
                    photoLinkButton.click()
                    time.sleep(2)

                    action = ActionChains(driver)
                    # this try-except is to handle (one photo but written more than one photo)
                    try:
                        ensureThatMoreThanOnePhoto=driver.find_element(By.XPATH,""" //*[@id="play"]/div/button[2]  """)
                    except:
                        closePhotoButton=driver.find_element(By.XPATH,""" //*[@id="image-header"]/div/button """)
                        closePhotoButton.click()
                        Photos[0]=driver.find_element(By.XPATH,""" //div[@class="RZ66Rb FgCUCc"]//img """).get_attribute("src")
                        fakeCodeToExitTheOuterTryExcept=driver.find_element(By.XPATH,"""  //:)  """)
                    
                    a_photos = driver.find_elements(By.XPATH, """ //div[@class="m6QErb "]//a """)
                    le = 0

                    while len(a_photos) < 30:
                        print(len(a_photos))
                        var = len(a_photos)
                        scroll_origin = ScrollOrigin.from_element(a_photos[len(a_photos)-1])
                        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
                        time.sleep(2)
                        a_photos = driver.find_elements(By.CLASS_NAME, "OKAoZd ")

                        if len(a_photos) == var:
                            le+=1
                            if le > 1:
                                break
                        else:
                            le = 0
                    print("-----------------------------")
                    print("    Photos Scrolling Done    ")
                    print("-----------------------------")
                    
                    PhotoLinks=driver.find_elements(By.XPATH,""" //div[@class="U39Pmb"] """)
                    counter=1
                    for links in PhotoLinks:
                        photoLink=PhotoLinks[counter-1].get_attribute("style")[23:-3]
                        
                        if(photoLink=="//:0" or photoLink==None):
                            links.click()
                            
                            
                        Photos[counter-1]=photoLink
                        counter=counter+1
                        print(photoLink)
                        if(counter>20):
                            break   
                    closePhotosTabButton=driver.find_element(By.XPATH,""" //*[@id="image-header"]/div/button """)
                    closePhotosTabButton.click()

                    
                except:
                    if(Photos[0]):
                        print("one photo found")
                    else:
                        print("No photos")
                
                
                
                time.sleep(2)

                #description
                try:
                    description=driver.find_element(By.XPATH,""" //span//button[@class="DkEaL "] """).text
                    print(description)
                    frfv=driver.find_elements(By.XPATH,"GFVCFV")
                except:
                    description=None


                #Address
                Address=divs[0].text  
                print(Address)                                                                                     


                #phone number
                phoneFound=False
                
                while( phoneFound==False):
                    for j in range(len(divs)):                                                                                 
                        for z in range(len(divs[j].text)):                                  #seeing every div from inside
                            if((str(divs[j].text)[z].isnumeric()) or (str(divs[j].text)[z] ==" ") or (str(divs[j].text)[z] == "+")):
                                if(z == (len(str(divs[j].text)))-1):                                           #does he tested all the char of the phone number or not
                                    phone  = divs[j].text
                                    phoneFound=True  
                                else:
                                    continue
                            else:
                                if(phoneFound==False):
                                    if(j == len(divs)-1):                                           # does he tested all the divs or not
                                        phone  = "not available"
                                        phoneFound=True                                            # to break the outer loop only (practically phone is not found )
                                    else:  
                                        break
                                
                print(phone)      
                  
                #Website
                websiteFound=False                                                                               
                for z in range(len(divs)):
                    if(len(divs[z].text)> 5):
                        if str(divs[z].text)[-4] == "." or str(divs[z].text)[-3] == "." or str(divs[z].text)[-5] == ".":         # www.  , .com , .site
                            if(str(divs[z].text)=="facebook.com") or (str(divs[z].text)=="m.facebook.com"):
                                website=driver.find_element(By.XPATH,""" //div[@class="RcCsl fVHpi w4vB1d NOE9ve M0S7ae AG25L "]/a  """).get_attribute("href")
                            else:
                                website = divs[z].text
                            websiteFound=True
                                
                        if(websiteFound==False):
                            website="Not available"
                    # else:
                    #     website="Not available"  
                    #     print("not av")
                print(website)
                
                # rating
                Rating_Html = soup.findAll('div', {"class": "F7nice"})
                rating=Rating_Html[0].text[0:3]
                noOfRatings=Rating_Html[0].text[4:-1]
                
                print("rating="+rating)
                print("no of rating="+noOfRatings)
                
                #location link
                location_Html=driver.find_element(By.XPATH,""" //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[5]/button """)
                location_Html.click()
                time.sleep(2)
                #the function of this loop is to wait until the link is loaded soo noo error occur while executing the code
                linkFound=False
                while(linkFound==False):
                    try:
                    # locationLink=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[3]/div/div/div[1]/div[4]/div[2]/input """).get_attribute("value")
                        locationLink=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[2]/div/div/div/div[3]/div[2]/div[2]/input """).get_attribute("value")
                        linkFound =True
                        break
                    except:
                        time.sleep(1)
                        continue
                    
                time.sleep(2)
                closeSharePanel=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/button """)
                closeSharePanel.click()
                time.sleep(2)
                print(locationLink)
                
                

                
                #plus code 
                counter=0
                plusCodeValue ="not available"
                plusCodeFound=False   
                divPhotos=driver.find_elements(By.XPATH,"""//div[@class="cXHGnc"]""")   
                
                #  replaced code with the old PlusCode scrapper version
                try:
                    if(driver.find_element(By.XPATH,""" //button[@data-tooltip="Copy plus code"] """)):
                        plusCodeValue=driver.find_element(By.XPATH,""" //button[@data-tooltip="Copy plus code"] //div[@class="rogA2c "]//div[@class="Io6YTe fontBodyMedium kR99db "] """).text
                    else:
                        plusCodeValue="not available"
                        
                except:
                    print("plus code:"+plusCodeValue)
    



                #old version plusCode scrapper ( not validate now)
                
                # while(plusCodeFound==False):
                    
                #     for z in range(len(divPhotos)):

                #         t1="(//div[@class='cXHGnc']/div/img)["
                #         t2=z+1
                #         t3="]"
                #         photoPath=str(t1)+str(t2)+str(t3)
                #         Photo=driver.find_element(By.XPATH,str(photoPath) )
                #         if(Photo.get_attribute("src")=="https://maps.gstatic.com/mapfiles/maps_lite/images/2x/ic_plus_code.png"):
                #             time.sleep(1)
                #             plusCodeValue=divs[counter].text
                #             plusCodeFound=True
                #             break
                #         else:
                #             if(plusCodeFound==False):
                #                 if(z == len(divPhotos)-1):                                           # does he tested all the divs or not
                #                     plusCodeValue  = "not available"
                #                     plusCodeFound=True                                            # to break the outer loop only (practically phone is not found )
                #                 else:  
                #                     counter =counter+1
                                    
                #     if(plusCodeValue=="Send to your phone" or plusCodeValue=="إرسال إلى هاتفك"):
                #         plusCodeValue="not available"

                #print("plus code:"+plusCodeValue)
        

        
                p2 = driver.current_window_handle                #(save the last place in the main page)
                #Longitude and latitude (from plus code if found or from address if not found)
                plusCodeToLongLatWebsite ="https://plus.codes/map"
                # Open a new window
                driver.execute_script("window.open('');")
                #close the certain window 
                driver.close()
                # Switch to the new window and open the website
                driver.switch_to.window(driver.window_handles[1])
                driver.get(plusCodeToLongLatWebsite)
                time.sleep(2)
                inputPlusCode = driver.find_element(By.XPATH, """ //*[@id="search-input"] """)
                
                if(plusCodeValue != "not available"):
                    time.sleep(0.5)
                    inputPlusCode.send_keys(plusCodeValue)
                    time.sleep(1)
                    inputPlusCode.send_keys(Keys.ENTER)
                else:
                    inputPlusCode.send_keys(Address)
                    time.sleep(1)
                    inputPlusCode.send_keys(Keys.ENTER)
                time.sleep(2)
                showLongitudeAndLatitudePanel=driver.find_element(By.XPATH,""" //*[@id="summary"]/div[1] """)
                showLongitudeAndLatitudePanel.click()
                
                time.sleep(2)
                
                longitudeAndLatitude =driver.find_element(By.XPATH,""" //*[@id="placecard-details"]/div[3]/div[1]  """).text
                latitude , longitude = longitudeAndLatitude.split(',')
                print("longitude="+longitude)
                print("latitude="+latitude)
                
                driver.close()
                time.sleep(1)
                print("reached1")
                driver.switch_to.window(p1)
                print("reached2")
                
                print([name ,Address , phone ,website , rating ,noOfRatings,locationLink,plusCodeValue ,latitude ,longitude , Photos[0], Photos[1], Photos[2], Photos[3], Photos[4], Photos[5], Photos[6], Photos[7], Photos[8], Photos[9], Photos[10], Photos[11], Photos[12], Photos[13], Photos[14], Photos[15], Photos[16], Photos[17], Photos[18], Photos[19]])
                record.append((name ,Address , phone ,website ,rating ,noOfRatings ,"Cairo",locationLink ,plusCodeValue ,latitude ,longitude , Photos[0], Photos[1], Photos[2], Photos[3], Photos[4], Photos[5], Photos[6], Photos[7], Photos[8], Photos[9], Photos[10], Photos[11], Photos[12], Photos[13], Photos[14], Photos[15], Photos[16], Photos[17], Photos[18], Photos[19],"resturants","Cairo",description))
                df=pd.DataFrame(record,columns=['Name' ,'Address' ,'Phone number' ,'Website' ,'Rating','Number of ratings' ,"Governorate","location Link","Plus code" ,"Latitude" ,"Longitude" ,"Photo 1","Photo 2","Photo 3","Photo 4","Photo 5","Photo 6","Photo 7","Photo 8","Photo 9","Photo 10","Photo 11","Photo 12","Photo 13","Photo 14","Photo 15","Photo 16","Photo 17","Photo 18","Photo 19","Photo 20" ,"Category","Region" ,"Place description"])  # writing data to the file
                df.to_csv(filename + '.csv',index=False,encoding='utf-8')
        except:
            driver.switch_to.window(p1)
            print("error")
            continue
        
        
        #the following lines of code must be edited if the lang is changed to arabic (English works)   
        #the following code asserts that the browser on the right (main page)  
        print (driver.current_url[0:35])
        print(link[0:35])
        if(driver.current_url[0:35] == str(link)[0:35]):
            continue
        else:
            driver.close()
            driver.switch_to.window(p1)




driver.get(str(link))
time.sleep(10)
Selenium_extractor()


5
10
15
20
25
30
35
40
45
50
----------------------
    Scrolling Done    
----------------------
ZAZA CUISINE by the Nile
20
-----------------------------
    Photos Scrolling Done    
-----------------------------
https://lh5.googleusercontent.com/p/AF1QipPMoLP60BFHq-qpIzjOXxVIHW8RLn1xC50la_Ni=w203-h109-k-no
https://lh5.googleusercontent.com/p/AF1QipMEkG9eCFcIskO6z83-ugl5nIlmw6pF7DUNk3e0=w203-h270-k-no
https://lh5.googleusercontent.com/p/AF1QipP-KqHjS9IZPVQlvKPUcxZtCPk3gr3-w_6DnqeO=w203-h136-k-no
https://lh5.googleusercontent.com/p/AF1QipMjX5dxtrINFsLSP-jiQbtDHwq2oxoNkRgwOpIx=w203-h114-k-no
//:0
https://lh5.googleusercontent.com/p/AF1QipNxW7wQCFsbMh9m0lJg5P6fspr6SpQntmGIolXP=w203-h218-k-no
https://lh5.googleusercontent.com/p/AF1QipOkTUyNjYrs78KZB91umhwEjqJ-AKCT1VDOSs5v=w203-h270-k-no
//:0
https://lh5.googleusercontent.com/p/AF1QipPHtpBFzOIpxN9Y7fVFvXmXhmYT-_uDXYUkWOJi=w203-h270-k-no
https://lh5.googleusercontent.com/p/AF1QipMJdo9hM1BqE2U2DOml8hJT2Oc3EipG87bXXPER=w203-h270-k-no
//:0
h